In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import norm, sem
from scipy.interpolate import UnivariateSpline
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from numpy.random import default_rng

import seaborn as sns
from pycebox.ice import ice, ice_plot
from matplotlib.cm import seismic

from sympy import *

In [2]:
x1,x2,x3,y = [],[],[],[]
N = 1000

for i in range(0,N):
    X1 = np.random.uniform(-1, 1, size = 1)
    X2 = np.random.uniform(-1, 1, size = 1)
    X3 = np.random.uniform(-1, 1, size = 1)
    Y = 0.2*X1 - 5*X2 + 10*X3

    x1.append(X1)
    x2.append(X2)
    x3.append(X3)    
    y.append(Y)

In [3]:
data = np.concatenate([x1,x2,x3,y], axis = 1)
df = pd.DataFrame(data, columns=['X1','X2','X3','Y'])

In [4]:
df

,X1,X2,X3,Y
0,0.887376,-0.441111,-0.450194,-2.118912
1,-0.380671,0.608813,-0.304792,-6.168122
2,-0.966732,0.042461,0.947461,9.068959
3,0.042492,-0.008547,-0.173544,-1.684207
4,-0.919086,-0.885070,-0.926882,-5.027288
...,...,...,...,...
995,0.436748,0.912386,0.288917,-1.585407
996,-0.719063,-0.388530,0.338177,5.180608
997,-0.823668,-0.341243,-0.058697,0.954514
998,-0.553968,-0.814603,-0.693372,-2.971496


In [5]:
X = df[['X1','X2','X3']]
X1, X2, X3 = df.X1, df.X2, df.X3
y = df.Y

std = {}

In [74]:
minus_X, plus_X, minus_y, plus_y = train_test_split(X, y, test_size=.5, random_state=42)
minus_df = minus_X.join(minus_y)
plus_df = plus_X.join(plus_y)

In [8]:
minus_df_new = change_features(minus_X, 0.05, 'minus')
plus_df_new = change_features(plus_X, 0.05, 'plus')

In [80]:
def change_features(X_og,X,i,alpha,group):
    X_list = []
    y_list = []

    for j in range(len(X)):
        new_values = {}
        std[i] = np.std(X[i])
        og_value = X[i].iloc[j]
        if group == 'plus':
            new_value = og_value + alpha*std[i]
        if group == 'minus':
            new_value = og_value - alpha*std[i]
        new_values[i] = new_value
        X_list.append(new_values)
    X_new = pd.DataFrame(X_list)
    X_new = X_new.join(X.reset_index().drop(columns=[i,'index']))
    for x in range(len(X_new)):
        y_list.append(0.2*X_new.X1[x] - 5*X_new.X2[x] + 10*X_new.X3[x]) 
    y_new = pd.DataFrame(y_list, columns=['Y'])
    df_new = X_new.join(y_new).append(X_og).reset_index().drop(columns=['index'])
    return(df_new)

In [81]:
plus_df_X1 = change_features(minus_df, plus_X, 'X1', 0.05, 'plus')
minus_df_X1 = change_features(plus_df, minus_X, 'X1', 0.05, 'minus')
minus_df_X1

,X1,X2,X3,Y
0,0.939992,0.068224,0.655162,6.398501
1,0.303338,0.144842,0.752496,6.861417
2,0.497681,0.744943,-0.404167,-7.666848
3,-0.578002,0.541020,-0.446609,-7.286787
4,0.894163,0.697276,-0.793092,-11.238467
...,...,...,...,...
995,0.998039,-0.230474,0.758924,8.941218
996,0.311169,-0.743893,-0.868578,-4.904082
997,-0.433357,-0.088391,-0.031418,0.041103
998,-0.070156,-0.030156,0.838900,8.525752


In [82]:
def cate(y, y_new):
    mean0 = np.mean(y)
    mean1 = np.mean(y_new)
    tao = mean1 - mean0
    return tao

In [84]:
minus_cate = {}
for x in minus_X:
    minus_df_new = change_features(minus_df, minus_X, x, 0.05, 'minus')
    c = cate(minus_y, minus_df_new.Y)
    minus_cate[x] = c
pd.DataFrame([minus_cate])

,X1,X2,X3
0,-0.002953,0.072105,-0.142519


In [85]:
plus_cate = {}
for x in plus_X:
    plus_df_new = change_features(minus_df, plus_X, x, 0.05, 'plus')
    c = cate(plus_y, plus_df_new.Y)
    plus_cate[x] = c
pd.DataFrame([plus_cate])

,X1,X2,X3
0,-0.004472,-0.078528,0.13318
